In [ ]:
!pip install selenium

In [ ]:
#IMPORTS
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
import zipfile
import io
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")

chrome_options = Options()
chrome_options.add_argument("--headless")
download_path = './'
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
column_names = ['size', 'uid', 'url']
df = pd.read_csv('/content/drive/MyDrive/CSCI-499 Project/csv', header=None, delimiter=' ', names=column_names)
df.head(5)

,size,uid,url
0,150383,297a16b493de7cf6ca809a7cc31d0b93,http://data.gdeltproject.org/gdeltv2/201502182...
1,318084,bb27f78ba45f69a17ea6ed7755e9f8ff,http://data.gdeltproject.org/gdeltv2/201502182...
2,10768507,ea8dde0beb0ba98810a92db068c0ce99,http://data.gdeltproject.org/gdeltv2/201502182...
3,149211,2a91041d7e72b0fc6a629e2ff867b240,http://data.gdeltproject.org/gdeltv2/201502182...
4,339037,dec3f427076b716a8112b9086c342523,http://data.gdeltproject.org/gdeltv2/201502182...


In [ ]:
condition = (df['url'].str.contains('.gkg', na=False)) & (df['url'].str.contains('v2/2019', na=False) | df['url'].str.contains('v2/2020', na=False) | df['url'].str.contains('v2/2021', na=False) | df['url'].str.contains('v2/2022', na=False))
df = df[condition]

In [ ]:
keywords = ['ECON']

In [ ]:
def filter_keywords(text):
    return [word for word in text.split(';') if any(keyword.lower() in word.lower() for keyword in keywords)]

In [ ]:
location = ['US', 'United States']

In [ ]:
def extract_csv_data(url, df_csv, prev_date):
    # driver = webdriver.Chrome(options=chrome_options)
    # driver.get(url)
    response = requests.get(url)
    zip_file_path = os.path.join(download_path, url.split('/')[-1])
    # print(zip_file_path.split('/')[1].split('.zip')[0])

    if response.status_code == 200:
        with open(zip_file_path, 'wb') as f:
            f.write(response.content)

        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            csv_file = zip_ref.open(zip_file_path.split('/')[1].split('.zip')[0])
            try:

                time_stamp_str = zip_file_path.split('/')[1].split('.zip')[0].split('.gkg')[0]
                timestamp = datetime.strptime(time_stamp_str, '%Y%m%d%H%M%S')
                date = timestamp.date()

                if str(date) != str(prev_date):
                  df_csv.to_csv('/content/drive/My Drive/CSCI-499 Project/CSV/fdata' + str(prev_date) + '.csv', index=False)
                  print(len(df_csv))
                  prev_date = date
                  df_csv = pd.DataFrame()
                  # count_csv += 1
                  print('saved')

                df_dummy = pd.read_csv(io.TextIOWrapper(csv_file), header=None, sep='\t', encoding='cp1252')
                required_columns = [4, 7, 8, 9]
                df_selected = df_dummy.iloc[:, required_columns]
                df_econ_themes = df_selected[df_selected[7].str.contains('ECON', na=False)]
                df_econ_themes = df_econ_themes[df_econ_themes[9].str.contains('|'.join(location), na=False)]

                df_econ_themes['ECON_Themes'] = df_selected[7].apply(lambda x: ';'.join([theme for theme in str(x).split(';') if 'ECON' in (theme)]))
                df_econ_themes['Theme_Count'] = df_econ_themes['ECON_Themes'].apply(lambda x: len(x.split(';')))
                df_econ_themes_final = df_econ_themes[[4, 'ECON_Themes', 'Theme_Count']].rename(columns={4: 'URL'})



                # df_main = df_selected[df_selected[7].str.contains('|'.join(keywords)) | df_selected[8].str.contains('|'.join(keywords))]

                # df_main.loc[:, 'TAGS'] = df_main[7].apply(filter_keywords)
                # df_main['TAGS'] = df_main[7].apply(filter_keywords)

                # 3, 7, 8,
                # df_main = df_main.drop([3, 7, 8, 18, 26], axis=1)
                # df_main = df_main.dropna(subset=[4], axis=0)
                # df_main = df_main.rename(columns={'4': 'URL'})

                # concat to df_csv
                df_csv = pd.concat([df_csv, df_econ_themes_final])

                # df_main.to_csv('/content/drive/My Drive/CSCI-499 Project/csv_/fdata' + zip_file_path.split('/')[1].split('.zip')[0].split('.gkg')[0] + '.csv', index=False)
                os.remove(zip_file_path)
                return df_csv, prev_date

            except UnicodeDecodeError:
                os.remove(zip_file_path)
                return df_csv, prev_date

    else:
        print("Failed to download the zip file")


In [ ]:
count = 0
df_csv = pd.DataFrame()
prev_date = datetime.strptime('20221112000000', '%Y%m%d%H%M%S').date()
for idx, row in df.iterrows():
    count += 1
    # if count > 200:
        # break
    # print(count)
    date = datetime.strptime(row['url'].split('.gkg')[0].split('/')[-1], '%Y%m%d%H%M%S').date()
    if date >= prev_date:
      df_csv, prev_date = extract_csv_data(row['url'], df_csv, prev_date)
    # if df_extracted is not None:
    #     df_main = pd.concat([df_main, df_extracted], axis=0)
# print("Number of csv files :: ", count_csv)
print("\n", count)

7968
saved
6177
saved
11272
saved
16306
saved
16886
saved
14208
saved
13593
saved
8239
saved
5489
saved
11846
saved
14607
saved
10755
saved
6846
saved
6178
saved
4353
saved
5126
saved
10328
saved
14604
saved
16616
saved
16138
saved
14258
saved
8578
saved
4928
saved
11993
saved
12803
saved
14481
saved
13944
saved
10940
saved
7011
saved
5133
saved
9899
saved
16355
saved
12757
saved
12024
saved
9311
saved
6457
saved
4248
saved
9643
saved
13435
saved
13768
saved
11893
saved
10492
saved
5903
saved
3515
saved
5697
saved
7447
saved
10477
saved
9281
saved
11091
saved

 137045
